### 예스 24 크롤링

In [89]:
# webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/mac/.wdm/drivers/chromedriver/mac64/126.0.6478.182/chromedriver-mac-x64/chromedriver'

In [95]:
from selenium import webdriver
browser = webdriver.Chrome()

In [91]:
url = 'https://google.com'
browser.get(url)

In [ ]:
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME, 'uU7dJb').text()


In [97]:
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [ ]:
# 1페이지의 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터
# browser.find_element(By.CLASS_NAME, 'gd_name') # element : 요소
datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # elements : 리스트

for i in datas:
    print(i.get_attribute('href'))

In [98]:
# 3페이지까지의 링크 데이터 전부 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24

import time

link_list = []
for i in range(1,4):
    print("*" * 10,f"현재 {i} 페이지 수집 중 입니다.", "*" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지 전체의 링크 데이터
    # browser.find_element(By.CLASS_NAME, 'gd_name') # element : 요소
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # elements : 리스트

    for i in datas:
        link = i.get_attribute('href')

        link_list.append(link)

    time.sleep(3)

print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
********** 현재 2 페이지 수집 중 입니다. **********
********** 현재 3 페이지 수집 중 입니다. **********
['https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/128133897', 'https://www.yes24.com/Product/Goods/126511412', 'https://www.yes24.com/Product/Goods/129043527', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/127094902', 'https://www.yes24.com/Product/Goods/124225606', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/123930880', 'https://www.yes24.com/Product/Goods/123930891', 'https://www.yes24.com/Product/Goods/126845471', 'https://www.yes24.com/Product/Goods/126590468', 'https://www.yes24.com/Product/Goods/126702038', 'https://www.yes24.com/Product/Goods/125295

In [64]:
len(link_list)

72

In [100]:
# for link in link_list:
#     # 상세 페이지로 이동
#     browser.get(link)

browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | "[0])
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | "[1])


In [101]:
text = "'| 판매지수 238,923 판매지수란?'"
text.split(" ")[2]

'238,923'

In [102]:
text = "'베스트 국내도서 1위 | 국내도서 1위 1주'"
text.split(" | ")[0]

"'베스트 국내도서 1위"

In [ ]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저장 (csv)
import pymysql 
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='!Passw0rd',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # 2024년 7월 22일 -> 2024-07-22
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2024-07-22"

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        try:
            # rating에서 콤마 제거 및 소수점 변환
            rating = float(rating.replace(",", ""))
        except ValueError:
            rating = 0.0

        try:
            review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
            review = int(review.replace(",", ""))
        except ValueError:
            review = 0

        try:
            sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
            sales = int(sales.replace(",", ""))
        except (IndexError, ValueError):
            sales = 0

        try:
            price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
            price = int(price.replace(",", ""))
        except ValueError:
            price = 0

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(" | ")

        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking_part = parts[0]
                ranking = int(''.join(filter(str.isdigit, ranking_part)))  # 숫자만 호출
            except ValueError:
                ranking = 0

            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = int(''.join(filter(str.isdigit, ranking_weeks_part.split()[-1])))
            except (IndexError, ValueError):
                ranking_weeks = 0

        sql = """INSERT INTO Books(
        title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
        )
        VALUES(
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()
        time.sleep(2)

browser.quit()
conn.close()

In [84]:
# 상세 페이지 이동 후 데이터 크롤링
title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | "[0])
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | "[1])

title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks


('너에게 들려주는 단단한 말',
 '김종원 저',
 '퍼스트펭귄',
 '2024년 07월 10일',
 '10.0',
 '10',
 '238,923',
 '18,000',
 ['베스트', '국내도서', '1위', '|', '국내도서', '1위', '1주'],
 ['베스트 국내도서 1위 ', ' 국내도서 1위 1주'])